<h1><span style='color:#8E5609;background:white'> IP Address Data Enrichment - Team Cymru</span></h1>

---

## Load Dataset

In [2]:
import os
import ipaddress
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, datetime
import dns.query, dns.message, dns.reversename, dns.resolver

# This is our source of questionable IP addresses
df_ip_address = pd.read_csv("../datasets/download/src_ip_address_failed_login_attempts.txt", delim_whitespace=1, names=['ip_count','src_ip'])


### Review Dataset Properties

In [3]:
df_ip_address

,ip_count,src_ip
0,1,102.91.5.112
1,1,103.100.4.46
2,1,103.155.210.207
3,1,103.39.30.30
4,1,103.86.19.113
...,...,...
1943,78,2409:4053:718:f0de::24d:e0ac
1944,80,2409:4055:4e95:c662::214b:7a11
1945,89,103.200.74.23
1946,102,190.142.113.207


In [4]:
df_ip_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948 entries, 0 to 1947
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ip_count  1948 non-null   int64 
 1   src_ip    1948 non-null   object
dtypes: int64(1), object(1)
memory usage: 30.6+ KB


## Query Team Cymru

In [5]:
# Extract ip_address column into list
ip_addr_lst = df_ip_address["src_ip"].tolist()

# Send list, perform reversing of octects, perform query
'''
- Build a list of reversed octect v4/v6 queries
  - dig +short 31.108.90.216.origin.asn.cymru.com TXT
  - dig +short 8.6.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.2.0.0.b.0.6.8.4.1.0.0.2.origin6.asn.cymru.com. TXT
- Query respective origin.asn.cymru.com and origin6.asn.cymru.com servers
- Record ASN,Country Code and RIR into list of dictionaries
'''

def cymru_dns_q(_ip_addrs):
    
    v4_reverse_dom = dns.name.from_text("origin.asn.cymru.com.")
    v6_reverse_dom = dns.name.from_text("origin6.asn.cymru.com.")
    peer_dom = dns.name.from_text("peer.asn.cymru.com.")
    asn_dom = ".asn.cymru.com"
    sys_resolvers = dns.resolver.get_default_resolver().nameservers
    
    lookup_list = []
    
    # Status / time tracking
    print(f'Start Time: {datetime.now()}')
    
    for ip in _ip_addrs:
        # IP query to get  AS number, BGP prefix, Country Code, RIR, Alocation Date
        query_ip = dns.reversename.from_address(ip, v4_reverse_dom, v6_reverse_dom)
        query_ip_mesg = dns.message.make_query(query_ip, dns.rdatatype.TXT)
        query_ip_response = dns.query.udp(query_ip_mesg, sys_resolvers[0])
        
        # Check rcode.
        if query_ip_response.rcode() == 0:

            query_ip_answer = str(query_ip_response.answer[0][0]).replace('"', "").split("|")
            
            # ASN query to obtain AS description
            query_asn = 'AS' + str(query_ip_answer[0]).strip() + asn_dom
            query_asn_mesg = dns.message.make_query(query_asn, dns.rdatatype.TXT)
            query_asn_response = dns.query.udp(query_asn_mesg, sys_resolvers[0])
            query_asn_answer = str(query_asn_response.answer[0][0]).replace('"', "").split("|")
        
            # Peer query
            query_peer = dns.reversename.from_address(ip, peer_dom, peer_dom)
            query_peer_mesg = dns.message.make_query(query_peer, dns.rdatatype.TXT)
            query_peer_response = dns.query.udp(query_peer_mesg, sys_resolvers[0])
            query_peer_answer = str(query_peer_response.answer[0][0]).replace('"', "").split("|")
        
            # Build list of dictionaries for the response
            # 0 - ASN, 1 - BGP Prefix, 2 - Country Code, 3 - RIR, 4 - Allocation Date, 5 - AS Description, 6 - AS Peers
            lookup_list.append({'ASN': query_ip_answer[0].strip(), 'BGP Prefix': query_ip_answer[1].strip(), 'Country Code': query_ip_answer[2].strip(), 
                                'RIR': query_ip_answer[3].strip(), 'Allocation Date': query_ip_answer[4].strip(), 'AS Description': query_asn_answer[4].strip(),
                                'AS Peers': query_peer_answer[0].strip()})
        else:
            # NaN is used as its pandas criteria for working with the dropna functionality
            # See https://pandas.pydata.org/docs/user_guide/missing_data.html#missing-data
            lookup_list.append({'ASN': np.NaN, 'BGP Prefix': np.NaN, 'Country Code': 'None', 
                                'RIR': 'None', 'Allocation Date': np.NaN, 'AS Description': 'None',
                                'AS Peers': np.NaN})
        
    
    return lookup_list
        
# Dataframe of queried data
df_ip_address_data = pd.DataFrame(cymru_dns_q(ip_addr_lst))

# Status / time tracking
print(f'End Time: {datetime.now()}')

Start Time: 2022-11-17 12:01:49.691774
End Time: 2022-11-17 12:05:04.827090


## Concatnate datasets

In [6]:
# Show 45 Rows, set 45 -> None for all rows
pd.set_option('display.max_rows', 15)

# Combine dataframes
'''
Here we want to combine the original df_ip_address dataframe with our new data. 
Since we queried in sequential order these dataframes are combined on axis 1 (column).
'''
all_data = pd.concat([df_ip_address,df_ip_address_data], axis=1)

### Review Dataset Properties

In [7]:
# Sort by ip_count decending.
all_data.sort_values('ip_count', ascending=True)

,ip_count,src_ip,ASN,BGP Prefix,Country Code,RIR,Allocation Date,AS Description,AS Peers
0,1,102.91.5.112,29465,102.91.5.0/24,NG,afrinic,2018-09-04,"VCG-AS, NG",16637
213,1,2409:4063:218e:9863:d451:7988:ade4:a150,55836,2409:4063:2000::/36,IN,apnic,2017-03-22,"RELIANCEJIO-IN Reliance Jio Infocomm Limited, IN",5511
212,1,2409:4061:68b:83b9:a5a6:ae86:2ad2:51f2,55836,2409:4061::/36,IN,apnic,2017-03-22,"RELIANCEJIO-IN Reliance Jio Infocomm Limited, IN",64049
211,1,2409:4061:687:46b7::11a1:b1,55836,2409:4061::/36,IN,apnic,2017-03-22,"RELIANCEJIO-IN Reliance Jio Infocomm Limited, IN",5511
210,1,2409:4061:4e91:885a:8489:7e9a:1cfa:9746,55836,2409:4000::/22,IN,apnic,2017-03-22,"RELIANCEJIO-IN Reliance Jio Infocomm Limited, IN",64049
...,...,...,...,...,...,...,...,...,...
1943,78,2409:4053:718:f0de::24d:e0ac,55836,2409:4053::/36,IN,apnic,2017-03-22,"RELIANCEJIO-IN Reliance Jio Infocomm Limited, IN",5511
1944,80,2409:4055:4e95:c662::214b:7a11,55836,2409:4055:4000::/36,IN,apnic,2017-03-22,"RELIANCEJIO-IN Reliance Jio Infocomm Limited, IN",64049
1945,89,103.200.74.23,NaN,NaN,None,None,NaN,None,NaN
1946,102,190.142.113.207,21826,190.142.112.0/21,VE,lacnic,2007-01-23,"Corporacion Telemic C.A., VE",263703


In [8]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948 entries, 0 to 1947
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ip_count         1948 non-null   int64 
 1   src_ip           1948 non-null   object
 2   ASN              1943 non-null   object
 3   BGP Prefix       1943 non-null   object
 4   Country Code     1948 non-null   object
 5   RIR              1948 non-null   object
 6   Allocation Date  1943 non-null   object
 7   AS Description   1948 non-null   object
 8   AS Peers         1943 non-null   object
dtypes: int64(1), object(8)
memory usage: 137.1+ KB


## Generate Enriched Dataset

In [8]:
all_data.to_csv('../datasets/generated/src_ip_failed_login_attempts_team_cymru_ip_to_asn_mapped.csv', index=False)

<div class="alert alert-block alert-success">
  <b>Next Steps: </b>
     With the new enriched dataset created you can move onto futher analysing the dataset for actionable content. 
</div>

---
<h4><span style='color:#8E5609;background:white'>www.threatlocus.com &copy</span></h4>

Copyright (C) 2022 www.threatlocus.com / Stephen Scally

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

---